### <center id="d1"><h1> 🦜🔗 📝 `MEMORY` - запускаем чат-режим!</h1></center>

### Оглавление
<img src='../images/memories_conv.png' align="right" width="528" height="428" style="border-radius: 0.75rem;" >
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#d1">🚀 Введение </a></li>
<li><a href="#d2">🧠 InMemoryChatMessageHistory - храним историю в памяти</a>
<li><a href="#d3">ChatPromptTemplate + память = цепочка всё помнит 🧠 </a>
<li><a href="#d4">🧸 Выводы и заключения ✅ </a>
    
</ul></font></p>

### 🧑‍🎓 В этом ноутбуке рассмотрим подробнее работу с `Memory` 

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
📖 Память позволяет LLM помнить предыдущие взаимодействия с пользователем. По умолчанию LLM является `stateless`, что означает независимую обработку каждого входящего запроса от других взаимодействий. На самом деле, когда мы вводим запросы в интерфейсе ChatGPT, под капотом работает память - т.е. модель отвечает нам, исходя из контекста всего диалога. 

Посмотрим как реализовать этот механизм в `LangChain`.

In [1]:
#!pip install langchain langchain-openai openai -q

In [5]:
import os
from getpass import getpass

In [3]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain_openai import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [6]:
# Если используете ключ из курса, запустите эту ячейку
from langchain_openai import ChatOpenAI

#course_api_key= "Введите API-ключ полученный в боте"
course_api_key = getpass(prompt='Введите API-ключ полученный в боте')

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini', 
                 base_url="https://aleron-llm.neuraldeep.tech/")

Введите API-ключ полученный в боте ········


   # <center id="d2">  🧠 `InMemoryChatMessageHistory` - храним историю в памяти

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
📖 Один из способов сохранять историю диалога в `LangChain` - это хранение контеста в оперативной памяти (`InMemoryChatMessageHistory`), подходит, например для сценариев, когда всё взаимодействие с пользователем происходит за один сеанс (в одном диалоге), после перезапуска всё обнулится.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Чтобы прикрутить к любому `Runnable` объекту память, потребуется обёртка `RunnableWithMessageHistory`. Объект `Runnable` - наиболее близок по смыслу к "запускаемый объект".

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Чтобы в памяти не путались диалоги от разных пользователей и по различным темам при запросе  в конфигурацию добавляется `sessiоn ID`. Можно добавить и несколько ключей, например ещё `user ID`, чтобы отельно идентифицировать несколько различных диалогов на разные темы с одним пользователем (вспомните как создаёте новый чат с ChatGPT).

In [7]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Создадим словарь, который будет мапить session_id с историей диалога этой сессии
store = {}

# Напишем функцию, которая возвращает историю диалога по session ID.
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [8]:
# Создаём конфиг для нашего Runnable, чтобы указать ему session_id при вызове
config = {"configurable": {"session_id": "1"}}

llm_with_history = RunnableWithMessageHistory(
    llm,
    get_session_history
)

# к вызову добавляем параметр config
llm_with_history.invoke("Привет, ChatGPT! Меня зовут Иван. Как дела?", config=config).content

'Привет, Иван! У меня всё отлично, спасибо за спрос. Как я могу помочь тебе сегодня?'

In [9]:
# В store появилась первая пара запрос-ответ, и ключ "1" - session_id
print(store['1'])

Human: Привет, ChatGPT! Меня зовут Иван. Как дела?
AI: Привет, Иван! У меня всё отлично, спасибо за спрос. Как я могу помочь тебе сегодня?


In [10]:
llm_with_history.invoke("Сможешь помочь мне в написании кода на Python?", config=config).content

'Конечно, я постараюсь помочь! Что именно тебе нужно сделать в коде на Python?'

In [11]:
llm_with_history.invoke("Как вывести на экран фразу 'Hello, world!'?", config=config).content

"Для вывода фразы 'Hello, world!' на экран в Python, используйте следующий код:\n\n```python\nprint('Hello, world!')\n```\n\nПросто скопируйте этот код в ваш редактор или интерпретатор Python, запустите его, и вы увидите фразу 'Hello, world!' на экране. Надеюсь, это поможет!"

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
**Видно, что благодаря запоминанию контекста, модель знала, что нам нужен ответ именно c синтаксисом языка Python.**

В `InMemoryChatMessageHistory` сообщения сохраняются парами `HumanMessage` - `AIMessage`.  </div>

In [12]:
print(store['1'])

Human: Привет, ChatGPT! Меня зовут Иван. Как дела?
AI: Привет, Иван! У меня всё отлично, спасибо за спрос. Как я могу помочь тебе сегодня?
Human: Сможешь помочь мне в написании кода на Python?
AI: Конечно, я постараюсь помочь! Что именно тебе нужно сделать в коде на Python?
Human: Как вывести на экран фразу 'Hello, world!'?
AI: Для вывода фразы 'Hello, world!' на экран в Python, используйте следующий код:

```python
print('Hello, world!')
```

Просто скопируйте этот код в ваш редактор или интерпретатор Python, запустите его, и вы увидите фразу 'Hello, world!' на экране. Надеюсь, это поможет!


In [13]:
# Очистить историю диалога сессии можно методом clear()
get_session_history('1').clear()
store

{'1': InMemoryChatMessageHistory(messages=[])}

In [14]:
llm_with_history.invoke("Как меня зовут?", config=config).content

'Простите, но я не знаю вашего имени.'

<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Модель не смогла ответить, так как очистили предыдущий диалог. 

**Контекст (историю) можно задавать вручную, подгружать из файла или БД.**

In [15]:
from langchain_core.messages import HumanMessage, AIMessage

get_session_history('1').clear()

# Допустим эти примеры мы взяли из базы данных
user_message = HumanMessage(content="Привет меня зовут Алерон?")
ai_message = AIMessage(content="Привет, Алерон. Чем могу помочь?")

# загрузим диалог в память методом add_messages, принимающим список сообщений
get_session_history('1').add_messages([user_message, ai_message])

store

{'1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Привет меня зовут Алерон?', additional_kwargs={}, response_metadata={}), AIMessage(content='Привет, Иван. Чем могу помочь?', additional_kwargs={}, response_metadata={})])}

In [16]:
llm_with_history.invoke("Как меня зовут?", config=config).content

'Извините за путаницу, Вас зовут Алерон. Как я могу помочь вам сегодня?'

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
**Видно, что модель верно ответила на вопрос, так как мы предварительно загрузили в память эту информацию.**
Таким образом, сохранив историю диалога в базе данных, можно её подгружать при следующем контакте с пользователем.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
📖 **Для подгрузки в память так же можно использовать следующие методы:**
* `add_message` - добавляет любой тип сообщения
* `add_ai_message` - добавляет сообщение типа `AIMessage`
* `add_user_message` - добавляет сообщение типа `HumanMessage`
* `aadd_messages` - асинхронное добавление сообщений

In [19]:
get_session_history('1').clear()
get_session_history('1').add_user_message('Hello, World!')
get_session_history('1').add_ai_message('Hi!')

print(store['1'])

Human: Hello, World!
AI: Hi!


# <center id="d3"> `ChatPromptTemplate` + память = цепочка всё помнит 🧠

<div class="alert alert-info">

<img src='../images/mh.png' align="up" style="border-radius: 0.75rem;" >
    
До этого мы вызывали только LLM, теперь давайте разберёмся как добавить память к простой цепочке, в которой есть шаблон промпта.

In [20]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Создадим шаблон промпта для ассистента
# Оставим в шаблоне "заглушку" - MessagesPlaceholder, в которую будет подставляться история диалога
prompt = ChatPromptTemplate.from_messages([
    ("system", "Ты полезный ассистент, отлично разбирающийся в {topic}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

In [21]:
# собираем простую цепочку
chain = prompt | llm # поговорим больше цепочках в следующем уроке

# добавляем сохранение истории взаимодействий
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history, 
    input_messages_key="question",  # указываем название переменой запроса
    history_messages_key="chat_history", # название переменной для истории из шаблона
)

chain_with_history.invoke(
    {"topic": "математика", "question": "Чему равен синус 30?"},
    config={"configurable": {"session_id": "2"}}).content

'Синус угла 30 градусов равен 0.5.'

In [22]:
chain_with_history.invoke(
    {"topic": "математика", "question": "А чему равен косинус?"},
    config={"configurable": {"session_id": "2"}}).content

'Косинус угла 30 градусов равен \\( \\frac{\\sqrt{3}}{2} \\) или примерно 0.866.'

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Из контекста модель поняла, что мы спрашиваем про косинус угла 30 градусов.

👀 Посмотрим на `store` - теперь по 2 разным ключам хранятся истории разных диалогов.

In [23]:
print(store['1'])

Human: Hello, World!
AI: Hi!


In [24]:
print(store['2'])

Human: Чему равен синус 30?
AI: Синус угла 30 градусов равен 0.5.
Human: А чему равен косинус?
AI: Косинус угла 30 градусов равен \( \frac{\sqrt{3}}{2} \) или примерно 0.866.


# <center id="p8"> 🧸 Выводы и заключения ✅ 

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
📖 **Недостатки `InMemoryChatMessageHistory`:**
* История хранится в оперативной памяти. При интенсивных диалогах и большом количестве пользователей память может закончиться
* При перезагрузке или сбое сервера - вся история исчезнет
* Чем длиннее история, тем больше токенов придется подавать на вход модели
* В случае платных моделей, это будет накладно по финансам
* Контекстное окно моделей тоже ограничено по количеству токенов

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Поэтому рекомендуется сохранять не всё подряд, а выделять основные сущности из диалога или периодически суммаризовать историю сообщений.

<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Для продакшен решений можно рассмотреть более практичные форматы хранения истории в `Langchain`, например, релизованы:
* `FileChatMessageHistory` - сохраняет историю взаимодействия сообщений в файл.
* `RedisChatMessageHistory` - сохраняет историю сообщений чата в базе данных Redis.
* `SQLChatMessageHistory` - сохраняет историю сообщений чата в базе данных SQL.
* `MongoDBChatMessageHistory` - в базе данных Mongo
* и многие [другие](https://python.langchain.com/api_reference/community/chat_message_histories.html)

<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
    
Например, это могло бы выглядеть так:
```python
engine = sqlalchemy.create_engine("sqlite:///database/chat_history.db")
SQLChatMessageHistory(session_id=session_id, connection=engine)
```